# Process tomography

In [4]:
import pylab as py
import qutip as qp
from pyquil.quil import Program
import pyquil.api as api
import pyquil.gates as gt
qvm = api.QVMConnection()
import math,cmath,itertools,scipy.linalg



In [5]:
"""
Definition of a class to represent a Pauli matrix
"""
class Pauli():
    def __init__(self, label, prefactor = 1):
        label = label.strip()
        assert isinstance(label,str), u"Variable 'label' must be a string"
        assert all([x.lower() in 'izxy' for x in label]), u"Characters must be in the set 'izxy'"
        self.label = label.lower()
        self.n_qubits = len(label)
        prefactor = complex(prefactor)
        self.prefactor = approximate(prefactor.real)+ 1j * approximate(prefactor.imag)
        
    def qutip_representation(self, qubit_order='rigetti'):
        """
        Get a qutip representation of the Pauli operator
        """
        assert qubit_order.lower() in ['rigetti','qinfo'], u"Variable 'qubit_order' must be 'rigetti' or 'qinfo'"
        if '_qutip_repr' in vars(self) and self._qutip_repr[1] == qubit_order:
            return self._qutip_repr[0]
        else:
            terms = [pauli_matrices[x.lower()] for x in self.label]
            if qubit_order == 'rigetti':
                rho = qp.tensor(terms[::-1])
            elif qubit_order == 'qinfo':
                rho = qp.tensor(terms[::+1])
            rho = self.prefactor * rho
            self._qutip_repr = (rho, qubit_order) 
            return rho
        

    def __mul__(self, other):
        label = []
        prefactor = self.prefactor * other.prefactor
        assert(self.n_qubits == other.n_qubits), u"Not allowed"
        for labA, labB in zip(self.label,other.label):
            if labA == labB:
                label.append('i')
            elif labA == 'i':
                label.append(labB)
            elif labB == 'i':
                label.append(labA)
            elif labA == 'z' and labB == 'x':
                    label.append('y')
                    prefactor *= +1j
            elif labA == 'z' and labB == 'y':
                    label.append('x')
                    prefactor *= -1j
            elif labA == 'x' and labB == 'z':
                    label.append('y')
                    prefactor *= -1j
            elif labA == 'x' and labB == 'y':
                    label.append('z')
                    prefactor *= +1j
            elif labA == 'y' and labB == 'z':
                    label.append('x')
                    prefactor *= +1j
            elif labA == 'y' and labB == 'x':
                    label.append('z')
                    prefactor *= -1j
        label = ''.join(label)
        new = Pauli(label,prefactor)
        return new

    def trace(self):
        if any([x != 'i' for x in self.label]):
            return 0
        else:
            return 2**self.n_qubits
    
    def expect(self, state):
        result = self.prefactor
        lookup = {'z': {'H': +1,'V': -1}, 'X': {'D': +1,'A': -1}, 'Y': {'R': +1,'L': -1}}
        for sigma, st in zip(self.label, state):
            if sigma != 'i':
                result *= lookup.get(sigma,{}).get(st,0)
                if abs(result) < 1e-10:
                    break
        return result
    
def approximate(number, values=[-1,0,1]):
    for val in values:
        if abs(number-val) < 1e-12:
            return val
    return number
        
pauli_matrices = {}
pauli_matrices['i'] = qp.qeye(2)
pauli_matrices['x'] = qp.sigmax()
pauli_matrices['y'] = qp.sigmay()
pauli_matrices['z'] = qp.sigmaz()

In [6]:
def build_design_matrix(n_qubits):
    """
    Create a design matrix for a process tomography
    """
    lines = []
#     for initial_state 
    
    
def iterator_initial_states(n_qubits):
    for index in itertools.product('HVDARL',repeat=n_qubits):
        yield ''.join(index[::-1])
        
def iterator_basis(n_qubits):
    for index in itertools.product('izxy',repeat=n_qubits):
        yield ''.join(index[::-1])
        
def iterator_basis_choice(n_qubits):
    for index in itertools.product('zxy',repeat=n_qubits):
        yield ''.join(index[::-1])

def iterator_observables(n_qubits, basis_choice):
    for index in itertools.product(range(2),repeat=n_qubits):
        yield ''.join(['i' if flag==0 else basis for basis,flag in zip(basis_choice,index)][::+1])

In [7]:
# def build_design_matrix_statetomo(n_qubits):
#     powerOfTwo = 2**n_qubits
#     design_matrix = 1j*py.zeros((6**n_qubits,4**n_qubits))
#     hadamard = scipy.linalg.hadamard(powerOfTwo)
#     hadamard = py.eye(powerOfTwo)
#     for k,measure_basis in enumerate(iterator_basis_choice(n_qubits)):
#         for j, measure_operator in enumerate(iterator_observables(n_qubits, measure_basis)):
#             measure = Pauli(measure_operator)
#             for s, rho_element in enumerate(iterator_basis(n_qubits)):
#                 element = Pauli(rho_element)
#                 term = (measure * element).trace()
#                 design_matrix[k*powerOfTwo+j , s] = term
#             design_matrix[(k*powerOfTwo):((k+1)*powerOfTwo),:] = py.dot(hadamard,design_matrix[(k*powerOfTwo):((k+1)*powerOfTwo),:])

#     return design_matrix / powerOfTwo
            
def build_design_matrix_statetomo(n_qubits):
    powerOfTwo = 2**n_qubits
    design_matrix = 1j*py.zeros((6**n_qubits,4**n_qubits))
    hadamard = scipy.linalg.hadamard(powerOfTwo)
    for k,measure_basis in enumerate(iterator_basis_choice(n_qubits)):
        submatrix = 1j*py.zeros((2**n_qubits,4**n_qubits))        
        for j, measure_operator in enumerate(iterator_observables(n_qubits, measure_basis)):
            measure = Pauli(measure_operator)
            for s, rho_element in enumerate(iterator_basis(n_qubits)):
                element = Pauli(rho_element)
                term = (measure * element).trace()
                submatrix[j,s] = term
        submatrix = py.dot(hadamard,submatrix)
        design_matrix[(k*powerOfTwo):((k+1)*powerOfTwo),:] = submatrix

    return design_matrix / powerOfTwo

            
            

In [8]:
def build_design_matrix_processtomo(n_qubits):
    powerOfTwo = 2**n_qubits
    matrices = []
    hadamard = scipy.linalg.hadamard(powerOfTwo)
    for initial_state in iterator_initial_states(n_qubits):
        matrix = 1j*py.zeros((6**n_qubits,16**n_qubits))
        for k,measure_basis in enumerate(iterator_basis_choice(n_qubits)):
            submatrix = 1j*py.zeros((2**n_qubits,16**n_qubits))        
            for j, measure_operator in enumerate(iterator_observables(n_qubits, measure_basis)):
                measure = Pauli(measure_operator)        
                for r,label_r in enumerate(iterator_basis(n_qubits)):
                    for s,label_s in enumerate(iterator_basis(n_qubits)):
                        operator = (Pauli(label_s) * measure) * Pauli(label_r)
                        expect = operator.expect(initial_state)
                        submatrix[j,r*4**n_qubits + s] = expect
            submatrix = py.dot(hadamard,submatrix)
            matrix[(k*powerOfTwo):((k+1)*powerOfTwo),:] = submatrix
        matrices.append(matrix)
    design_matrix = 1j*py.zeros((36**n_qubits,16**n_qubits))

    counter = 0
    for index, submatrix in enumerate(matrices):
        design_matrix[counter:counter+len(submatrix),:] = submatrix
        counter += len(submatrix)
            
    return design_matrix / powerOfTwo




In [22]:
def save_to_csv(matrix, filename):
    csv_string = ''
    this = []
    for line in matrix:
        for number in line:
            if abs(number)>1e-12:
                this.append('{0:+.3f}{1}{2:.3f}j'.format(number.real,'+-'[number.imag < 0],abs(number.imag)))
            else:
                this.append('0')
        csv_string += ','.join(this) + '\n'
    with open(filename,'w') as fl:
        fl.write(csv_string)
    return csv_string

In [24]:
save_to_csv(p1, 'ProcessTomo1.csv')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  import sys


'+1.000+0.000j,+1.000+0.000j,0,0,+1.000+0.000j,+1.000+0.000j,0,0,0,0,0,0,0,0,0,0\n+1.000+0.000j,+1.000+0.000j,0,0,+1.000+0.000j,+1.000+0.000j,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,+1.000+0.000j,+0.000-1.000j,0,0,+0.000+1.000j,+1.000+0.000j\n+1.000+0.000j,+1.000+0.000j,0,0,+1.000+0.000j,+1.000+0.000j,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,+1.000+0.000j,+0.000-1.000j,0,0,+0.000+1.000j,+1.000+0.000j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.000+0.500j,+0.000+0.500j,+0.000+0.500j,+0.500+0.000j\n+1.000+0.000j,+1.000+0.000j,0,0,+1.000+0.000j,+1.000+0.000j,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,+1.000+0.000j,+0.000-1.000j,0,0,+0.000+1.000j,+1.000+0.000j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.500+0.000j,+0.500+0.000j,+0.500+0.000j,+0.000-0.500j,+0.000+0.500j,+0.000+0.500j,+0.000+0.500j,+0

In [32]:
qvm.wavefunction(Program([gt.X(10),gt.Z(11)])).amplitudes

array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j])